<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Import-Packages" data-toc-modified-id="Import-Packages-1">Import Packages</a></span></li></ul></div>

## Import Packages 

In [1]:
import os, sys
import numpy as np
import pandas as pd

import pickle

import matplotlib.pyplot as plt

from igraph.drawing.text import TextDrawer
import cairo

In [ ]:
# import_nbmodule used for import package in .ipynb
import import_nbmodule

# extract the exp folder path
currfolder = os.getcwd()
codefolder = currfolder[0 : currfolder.find('code')+len('code')]


# import graphVisual in connAnalyTool/graphVisual.pynb 
from connAnalyTool import graphVisual


# import threshold_ciCOH_normal function in simulated/ciCOH_threshold.pynb 
from simulated.ciCOH_threshold import threshold_ciCOH_normal

In [ ]:
def ciCOH_visual_save(ciCOH, chnInf_brainarea, chnInf_coord, chnInf_recchni, lowweight, savefile, texts, 
                      edgeWider = False, edgeWiderMul = 2):
    """
        
        Args:
            
            ciCOH (np.sdarray): ciCOH matrix (nchns, nchns)
            
            chnInf_brainarea (list): a list of string representing brain area for each channel, len(chn_area) = nchns
            
            chnInf_coord (numpy.ndarray): x, y coordinates for each channel, shape = (nchns, 2)
            
            chnInf_recchni(list): showed recording chni for each channel, shape = (nchns, )
            
            lowweight: the threshold lowweight, only weight>lowweight is treated as connection
            
            savefile: file to save the visualized figure
            
            texts: dict, the texts shown with positions, e.g. {'VPLo': [20, 40, 20], 'VLo': [20, 120, 20]}
            
            edgeWider: whether to wider the edger (default: False)
            
            edgeWiderMul: if edgeWider = True, this value is used for wider the edge width
            
        Output:
            the visualizaton of ciCOH is saved in savefile
            
    """
    
    weight = abs(ciCOH)

    # weight > lowweight
    weight[np.where(weight<lowweight)] =0
    
    graph = graphVisual.graph_create(weight, vsarea = chnInf_brainarea, 
                                     vscoord = chnInf_coord, vschni = chnInf_recchni)

    # set graph visualization style
    visual_style = dict()
    if edgeWider:
        visual_style['edge_width'] = [w * edgeWiderMul for w in graph.es['weight']]
    
    visual_style = graphVisual.graph_style(graph, visual_style)
    

    igplot = graphVisual.graph_plot(graph, visual_style, texts = texts)
    
    igplot.save(savefile)
    
    return igplot

In [ ]:
def unwanted_chandata_del(ciCOH, areas_unwanted, chnInf_recchni, chnInf_brainarea, chnInf_coord):
    """
        delete unwanted channels from ciCOH matrix
        
        @paras:
            ciCOH:
            
            areas_unwanted:
            
            chnInf_recchni(list): showed recording chni for each channel, shape = (nchns, )
            
            chnInf_brainarea (list): a list of string representing brain area for each channel, len(chn_area) = nchns
            
            chnInf_coord (numpy.ndarray): x, y coordinates for each channel, shape = (nchns, 2) 
        
    """
    
    chns_unwanted = []
    for area_unwanted in areas_unwanted:
        for i, area in enumerate(chnInf_brainarea):
            if area == area_unwanted:
                chns_unwanted.append(i)
    
    nchns = ciCOH.shape[0]
    ciCOH = np.delete(ciCOH, chns_unwanted, axis = 0)
    ciCOH = np.delete(ciCOH, chns_unwanted, axis = 1)
    
    
    chnInf_recchni = [i for j, i in enumerate(chnInf_recchni) if j not in chns_unwanted]
    chnInf_brainarea = [i for j, i in enumerate(chnInf_brainarea) if j not in chns_unwanted]
    chnInf_coord = [i for j, i in enumerate(chnInf_coord) if j not in chns_unwanted]

    return ciCOH, chnInf_recchni, chnInf_brainarea, chnInf_coord